# WebCam을 통한 얼굴 인식 및 스티커 부착

In [9]:
import cv2
import dlib 
import sys
import numpy as np

In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

cam = cv2.VideoCapture(0)
color_green = (255,255,0)
line_width = 2

In [11]:
# https://pysource.com/2020/04/10/add-watermark-to-multiple-images-with-opencv-and-python/

sticker1 = cv2.imread("red_ribbon.png", -1)
h_logo, w_logo, _ = sticker1.shape

sticker2 = cv2.imread("ironman.png", -1)
h_logo, w_logo, _ = sticker2.shape

sticker3 = cv2.imread("rabbit.png", -1)
h_logo, w_logo, _ = sticker3.shape

In [12]:
def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [13]:
toggle1 = 0 
toggle2 = 0
toggle3 = 0
toggle4 = 0
toggle5 = 0

center_y = 0
center_x = 0
top_y = 0
left_x = 0
bottom_y = 0
right_x = 0

In [14]:
while True:
    ret_val, img = cam.read()
    img = cv2.flip(img, 1)      # 좌우반전
    rgb_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dets = detector(rgb_image)
    logo = cv2.imread("red_ribbon.png", -1)
    logo2 = cv2.imread("ironman.png", -1)    
    logo3 = cv2.imread("rabbit.png", -1) 
    
    # 출력
    for det in dets:
        # 사각형 출력
        if(toggle2 == 1):
            cv2.rectangle(img,(det.left(), det.top()), (det.right(), det.bottom()), color_green, line_width)

        # 렌드마크 출력
        dlib_shape = predictor(img, det)
        shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])
        
        if(toggle1 == 1):
            for s in shape_2d:
                cv2.circle(img, center=tuple(s), radius=1, color=(255,255,255), thickness=0, lineType=cv2.LINE_AA)      
    

        h_img, w_img, _ = img.shape

        #############################################################################
        # 스티커1 처리 
        if(toggle3 == 1):        

            size_x = int(det.right()) - int(det.left())
            size_y = int((int(det.bottom()) - int(det.top()))/2)

            sticker1 = cv2.resize(logo, dsize=(size_x, size_y), interpolation=cv2.INTER_AREA)

            h_logo, w_logo, _ = sticker1.shape

            center_x = int((det.left() + det.right()) / 2)
            center_y = int((det.top() + det.bottom()) / 2)
            
            top_y = int(det.top()) - h_logo
            left_x = center_x - int(w_logo / 2)
            bottom_y = int(det.top())     
            right_x = left_x + w_logo   

            if(left_x>0 and right_x<w_img):
                if(top_y>0 and bottom_y<h_img):
                    # Get ROI
                    roi = img[top_y: bottom_y, left_x: right_x]
                    img = overlay_transparent(img, sticker1, left_x, top_y)
                    
                    
        #############################################################################    
        # 스티커2 처리 
        if(toggle4 == 1):        

            scale = 1.5
            size_x = int(det.right()*scale) - int(det.left()*scale)
            size_y = int(det.bottom()*scale) - int(det.top()*scale)

            sticker2 = cv2.resize(logo2, dsize=(size_x, size_y), interpolation=cv2.INTER_AREA)

            h_logo2, w_logo2, _ = sticker2.shape

            center_x = int((det.left() + det.right()) / 2)
            center_y = int((det.top() + det.bottom()) / 2)
            
            left_x =  center_x - int(size_x/2)
            top_y = center_y - int(size_y/2)
            
            right_x =  left_x + size_x
            bottom_y = top_y + size_y
            
            if(left_x>0 and right_x<w_img):
                if(top_y>0 and bottom_y<h_img):
                    # Get ROI
                    roi = img[top_y: bottom_y, left_x: right_x]
                    img = overlay_transparent(img, sticker2, left_x, top_y)    
         
        
        #############################################################################                
        # 스티커3 처리 
        if(toggle5 == 1):        

            size_x = int(det.right()) - int(det.left())
            size_y = int((int(det.bottom()) - int(det.top()))/2)

            sticker3 = cv2.resize(logo3, dsize=(size_x, size_y), interpolation=cv2.INTER_AREA)

            h_logo, w_logo, _ = sticker3.shape

            center_x = int((det.left() + det.right()) / 2)
            center_y = int((det.top() + det.bottom()) / 2)
            
            top_y = int(det.top()) - int(h_logo * 1.3)
            left_x = center_x - int(w_logo / 2)
            bottom_y = int(det.top())     
            right_x = left_x + w_logo   

            if(left_x>0 and right_x<w_img):
                if(top_y>0 and bottom_y<h_img):
                    # Get ROI
                    roi = img[top_y: bottom_y, left_x: right_x]
                    img = overlay_transparent(img, sticker3, left_x, top_y)
                      
                    

        ##########################################################################################    
        # debug print
        cv2.putText(img, "press ESC to exit", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))
        cv2.putText(img, "press 1 to show landmarks", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))
        cv2.putText(img, "press 2 to show rectangle", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0)) 
        cv2.putText(img, "press 3 to show ribbon", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))  
        cv2.putText(img, "press 4 to show ironman", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))  
        cv2.putText(img, "press 5 to show rabbit", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))  
               
            
        cv2.putText(img, "[Recognized Face] ", (500, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))

        cv2.putText(img, "left   : ", (500, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, "top    : ", (500, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, "right  : ", (500, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, "bottom : ", (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))

        cv2.putText(img, str(det.left()), (580, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, str(det.top()), (580, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, str(det.right()), (580, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        cv2.putText(img, str(det.bottom()), (580, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
        ##########################################################################################

        if(toggle3 ==1 or toggle4 ==1 or toggle5 ==1):
            ##########################################################################################
            # debug print
            cv2.putText(img, "[Add-on Picture] ", (500, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))

            cv2.putText(img, "left_x   : ", (500, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, "top_y    : ", (500, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, "right_x  : ", (500, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, "bottom_y : ", (500, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, "center_x  : ", (500, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, "center_y : ", (500, 260), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))

            cv2.putText(img, str(left_x), (580, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, str(top_y), (580, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, str(right_x), (580, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, str(bottom_y), (580, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, str(center_x), (580, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))
            cv2.putText(img, str(center_y), (580, 260), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0))    
            ##########################################################################################
    


    # press ESC to exit
#    if cv2.waitKey(1) == 27:
#        break  # esc to quit
        
    ret = cv2.waitKey(1)
    print('pressed key is {0}'.format(ret))
    if ret == 27:
        break
    elif ret == 49:
        if toggle1 == 0:
            toggle1 = 1
        elif toggle1 == 1:
            toggle1 = 0
        #st1 = str(toggle1)
        #cv2.putText(img, st, (500, 340), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))
    elif ret == 50:
        if toggle2 == 0:
            toggle2 = 1
        elif toggle2 == 1:
            toggle2 = 0
        #st2 = str(toggle2)
        #cv2.putText(img, st2, (500, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))
    elif ret == 51:
        if toggle3 == 0:
            toggle3 = 1
        elif toggle3 == 1:
            toggle3 = 0
        #st2 = str(toggle2)
        #cv2.putText(img, st2, (500, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))
    elif ret == 52:
        if toggle4 == 0:
            toggle4 = 1
        elif toggle4 == 1:
            toggle4 = 0       
    elif ret == 53:
        if toggle5 == 0:
            toggle5 = 1
        elif toggle5 == 1:
            toggle5 = 0       
            
            
    # show frame
    cv2.imshow('my webcam', img)
    
        
cv2.destroyAllWindows()

pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is 49
pressed key is -1
pressed key is -1
pressed key is -1
pressed key is -1
pressed ke